# Initial Insider Transactions

This notebook shows how to use edgartools to get access to initial insider transaction data

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/Initial-Insider-Transactions.ipynb)

In [ ]:
!pip install edgartools

In [1]:
from edgar import *
import pandas as pd
from tqdm.auto import tqdm
from rich import inspect, print
pd.options.display.max_columns = None

set_identity('Mike.Gardhouse@winery.com')

## Getting Insider Filings

To list insider filings on Edgar use `get_filings` and filter by form.

The forms are 

- **Form 3** - Initial ownership
- **Form 4** - Changes in ownership
- **Form 5** - Annual changes in ownership

In [3]:
filings = get_filings(form=4).head(10)
filings

╭────────────────────────────────────────────────── SEC Filings ──────────────────────────────────────────────────╮
│                                                                                                                 │
│    Form             CIK   Ticker   Company                                  Filing Date   Accession Number      │
│  ─────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│    4            1770787   TXG      10x Genomics, Inc.                       2025-03-04    0001610717-25-0000…   │
│    4            1555279   MASS     908 Devices Inc.                         2025-03-04    0001562180-25-0020…   │
│    4            1591588   AMRK     A-Mark Precious Metals, Inc.             2025-03-04    0001593968-25-0002…   │
│    4            1591588   AMRK     A-Mark Precious Metals, Inc.             2025-03-04    0001593968-25-0002…   │
│    4            1591588   AMRK     A-Mark Precious Metals, Inc.       

## Insider Filings for a Company

If you are interested in insider transactions for a specific company first find the company using ticker or cik, and from there you can find the insider transactions.

Let's look at insider information for **Vertex Pharmaceuticals**

In [4]:
company = Company("VRTX")
company

╭─────────────────────────────── 🏢 VERTEX PHARMACEUTICALS INC / MA [875320] VRTX ────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│ ╭───────────────────────────────────────────────── 📋 Entity ─────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ │   CIK   875320   Type   Operating   ○                                                                       │ │
│ │                                                                                                             │ │
│ │                                                                                                             │ │
│ │   Category                  Industry                            Fiscal

## Initial Insiders

The form 3 filings contain information about the initial positions for insiders in a company. This also means we can see roughly when persons became insiders from their initial position filings.

Let's see all the company's initial filings

In [5]:
initial_filings = (company.get_filings(form=3)
                     .filter(filing_date=':2025-03-04')) # Filter up to this date to keep the notebook data stable
initial_filings

╭───────────────────────────── Filings for VERTEX PHARMACEUTICALS INC / MA [875320] ──────────────────────────────╮
│                                                                                                                 │
│    #   Form       Description                                          Filing Date   Accession Number           │
│  ─────────────────────────────────────────────────────────────────────────────────────────────────────────      │
│    0   3          Initial statement of beneficial ownership            2024-05-17    0000875320-24-000146       │
│    1   3          Initial statement of beneficial ownership            2023-12-06    0000875320-23-000050       │
│    2   3          Initial statement of beneficial ownership            2023-10-10    0001209191-23-052532       │
│    3   3          Initial statement of beneficial ownership            2023-09-05    0001209191-23-048157       │
│    4   3          Initial statement of beneficial ownership           

There are just over 50 initial insider filings since **2004** for **VRTX**. This could be related to the SEC's **2003** rule change mandating electronic filings with previous filings being paper based. 

## Extracting the initial insider information

To get access to the data inside the insider filings we need to loop through each filing, convert to data objects, then dataframes. Then we concatenate into a final dataframe.

In [6]:
insider_data = []

for filing in tqdm(initial_filings):
    # Convert to data object
    form3 = filing.obj()
    # Get the data in a dataframe. Set detailed = False to get 1 summary row
    df = form3.to_dataframe(detailed=False)

    insider_data.append(df)

insider_data = (pd.concat(insider_data, ignore_index=True)
                    .drop(columns=['Form', 'Issuer', 'Ticker', 'Remarks',
                                   'Has Derivatives', 'Derivative Holdings']) 
                    .reset_index(drop=True)
                    
               )
insider_data

  0%|          | 0/52 [00:00<?, ?it/s]

,Date,Insider,Position,Total Shares,Holdings,Common Stock Holdings
0,2024-05-15,Jennifer Schneider,Director,0,0,0
1,2023-12-05,Nancy Thornberry,Director,0,0,0
2,2023-10-05,Michel Lagarde,Director,0,0,0
3,2023-08-29,Edward Morrow Atkinson III,"EVP, Chief Technical Ops. Off.",13879,1,1
4,2022-09-19,Jonathan Biller,EVP and Chief Legal Officer,0,0,0
5,2022-05-18,Suketu Upadhyay,Director,0,0,0
6,2021-05-01,Kristen Ambrose,SVP & Chief Accounting Officer,1394,1,1
7,2021-03-02,Joy Liu,"SVP, General Counsel",13394,4,1
8,2021-02-03,Bastiano Sanna,"EVP, Cell & Genetic Therapies",21175,1,1
9,2020-06-03,Diana Mckenzie,Director,207,1,1


In [7]:
initial_filings[34]

╭──────── Form 3 VERTEX PHARMACEUTICALS INC / MA [875320] VRTX ─────────╮
│                                                                       │
│   Accession Number       Filing Date   Period of Report   Documents   │
│  ───────────────────────────────────────────────────────────────────  │
│   0000875320-09-000055   2009-07-07    2009-07-06         2           │
│                                                                       │
│                                                                       │
│                                                                       │
│                                                                       │
╰────────────── Initial statement of beneficial ownership ──────────────╯

## Viewing specific insider filings

#### Jennifer Schneider (Director)
Jennifer Schneider's initial filing indicated no holdings

In [8]:
initial_filings[0].obj()

╭──────────────── Initial Beneficial Ownership ────────────────╮
│ Insider:  Jennifer Schneider                                 │
│ Position: Director                                           │
│ Company:  VERTEX PHARMACEUTICALS INC / MA (VRTX)             │
│ Date:     2024-05-15                                         │
│ Form:     Form 3 (Initial Statement of Beneficial Ownership) │
│ No Securities Beneficially Owned                             │
│ Remarks: Exhibit 24 - Power of Attorney                      │
╰──────────────────────────────────────────────────────────────╯

#### Edward Morrow Atkinson III (EVP)

Here we see **Edward Morrow Atkinson III** initial position in common stock

In [9]:
initial_filings[3].obj()

╭──────────────── Initial Beneficial Ownership ────────────────╮
│ Insider:  Edward Morrow Atkinson III                         │
│ Position: EVP, Chief Technical Ops. Off.                     │
│ Company:  VERTEX PHARMACEUTICALS INC / MA (VRTX)             │
│ Date:     2023-08-29                                         │
│ Form:     Form 3 (Initial Statement of Beneficial Ownership) │
│         Common Stock Holdings                                │
│                                                              │
│   Security       Shares   Ownership                          │
│  ───────────────────────────────────                         │
│   Common Stock   13,879   Direct                             │
│                                                              │
│ Remarks: Exhibit 24 - Power of Attorney                      │
╰──────────────────────────────────────────────────────────────╯

#### Ourania Tatsis
This filing by Ourania Tatsis has both Common Stock and Derivative Holdings

In [10]:
initial_filings[10].obj()

╭────────────────────────────────── Initial Beneficial Ownership ───────────────────────────────────╮
│ Insider:  Ourania Tatsis                                                                          │
│ Position: SVP, CRO                                                                                │
│ Company:  VERTEX PHARMACEUTICALS INC / MA (VRTX)                                                  │
│ Date:     2019-10-01                                                                              │
│ Form:     Form 3 (Initial Statement of Beneficial Ownership)                                      │
│          Common Stock Holdings                                                                    │
│                                                                                                   │
│   Security          Shares   Ownership                                                            │
│  ──────────────────────────────────────                                         